## Caso de estudio
Para mi caso de estudio elegí el caso de entrenar multiples regresiones lineales. Elegí este caso porque a veces cuando no conocemos el modelo que mejor predice algún fenómeno entrenamos muchos modelos a veces con distintos parámetros y realizamos una evaluación cruzada para elegir el mejor modelo basándonos en esta evaluación.


In [41]:
import time
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import pandas as pd

# Cargamos el dataset California Housing
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Entrenamos múltiples modelos secuencialmente
def entrenar_modelo_secuencial(X, y, num_modelos=1500000):
    modelos = []
    for i in range(num_modelos):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        modelo = LinearRegression().fit(X_train, y_train)
        modelos.append(modelo)
    return modelos

# Medimos el tiempo de ejecución secuencial
start_time = time.time()
modelos_secuenciales = entrenar_modelo_secuencial(X, y, num_modelos=5)
end_time = time.time()

print("Tiempo de ejecución secuencial:", end_time - start_time, "segundos")



Tiempo de ejecución secuencial: 0.25035643577575684 segundos


La porción anterior del código se ejecutó de manera secuencial, es decir, cada modelo se entrenaba uno detrás de otro. El proceso tardó 0.25 segundos, esto fue muy rápido porque la tarea en esta cantidad sigue siendo simple entonces que se resuelva de manera directa puede ser efectivo.

In [42]:
import threading

# Función para entrenar un modelo en un hilo
def entrenar_modelo_hilos(X, y, resultado, indice):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    modelo = LinearRegression().fit(X_train, y_train)
    resultado[indice] = modelo

# Entrenamos múltiples modelos usando hilos
def entrenar_modelos_con_hilos(X, y, num_modelos=1500000):
    resultados = [None] * num_modelos
    hilos = []

    for i in range(num_modelos):
        hilo = threading.Thread(target=entrenar_modelo_hilos, args=(X, y, resultados, i))
        hilos.append(hilo)
        hilo.start()

    # Esperamos a que todos los hilos terminen
    for hilo in hilos:
        hilo.join()

    return resultados

# Medimos el tiempo de ejecución con hilos
start_time = time.time()
modelos_hilos = entrenar_modelos_con_hilos(X, y, num_modelos=5)
end_time = time.time()

print("Tiempo de ejecución con multi-hilos:", end_time - start_time, "segundos")



Tiempo de ejecución con multi-hilos: 0.24039816856384277 segundos


La porción de código anterior ejecuta de manera concurrente con multi-hilos, lamentablemente, es importante especificar que python obedece el GIL, lo que significa que para el multihilos solo tiene como recurso un núcleo, limitando el poder de procesamiento de la tarea. Se ejecutó en 0.24 que es ligeramente más rápido que entrenar los modelos de manera secuencial, esto porque aunque solo lo haga con un núcleo, sí se ejecuta con múltiples hilos y esto fue efectivo en este caso.

In [43]:
import multiprocessing

# Función para entrenar un modelo en un proceso
def entrenar_modelo_proceso(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    modelo = LinearRegression().fit(X_train, y_train)
    return modelo

# Entrenamos múltiples modelos usando procesos
def entrenar_modelos_con_procesos(X, y, num_modelos=1500000):
    with multiprocessing.Pool(processes=num_modelos) as pool:
        resultados = pool.starmap(entrenar_modelo_proceso, [(X, y) for _ in range(num_modelos)])
    return resultados

# Medimos el tiempo de ejecución con procesos
if __name__ == "__main__":
    start_time = time.time()
    modelos_procesos = entrenar_modelos_con_procesos(X, y, num_modelos=5)
    end_time = time.time()

    print("Tiempo de ejecución con multi-procesos:", end_time - start_time, "segundos")


Tiempo de ejecución con multi-procesos: 0.5181884765625 segundos


Para esta última parte de código se ejecutó de manera concurrente con múltiples procesos, la ventaja de este efoque es que no importa el GIL, de esta manera al crear más procesos sí se pueden procesar con múltiples núcleos, lo que significa que al procesar es más efectivo, sin embargo, hacer los nuevos procesos toma recursos que hacen que el tiempo de ejecución sea más lento, lo que hace que esta porción sea la más lenta de ejecutar.

## Conclusión
El enfoque más efectivo fue el de multi-hilos, sin embargo, es importante notar que la diferencia entre este enfoque y el secuencial fue muy poca, lo que nos habla de que probablemente es una tarea básica en la que el tiempo "extra" que requiere ejecutar el problema con concurrencia no vale la pena debido a que la resolución directa del problema es lo suficientemente rápida. Esto también es verdad para cuando se ejecuta con el enfoque de multiproceso ya que en ese caso generar los procesos toma muchos recursos y tarda mucho tiempo. Así concluyo que el problema es muy sencillo y por eso el enfoque más rápido fue el secuencial.
